## Package Import 

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import time

from keras.callbacks import LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping
from functools import partial
from sklearn.model_selection import train_test_split

In [ ]:
def plot_history(model):
    f,ax=plt.subplots(2,1,figsize=(10,10)) 

    # training loss and validation loss
    ax[0].plot(model.history.history['loss'],color='b',label='Training Loss')
    ax[0].plot(model.history.history['val_loss'],color='r',label='Validation Loss')

    # training accuracy and validation accuracy
    ax[1].plot(model.history.history['accuracy'],color='b',label='Training  Accuracy')
    ax[1].plot(model.history.history['val_accuracy'],color='r',label='Validation Accuracy')

    plt.legend()

## Load Dataset

### Load dataset from Keras

In [ ]:
# import the dataset from keras
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

In [ ]:
# split the train set into train set and validation set
x_train, x_validation, y_train, y_validation = train_test_split(x_train,y_train,test_size=.2)

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
validation_ds = tf.data.Dataset.from_tensor_slices((x_validation, y_validation))
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))

## AlexNet

In [ ]:
def process_images(image, label):
    # Normalize images to have a mean of 0 and standard deviation of 1
    image = tf.image.per_image_standardization(image)
    # Resize images from 32x32 to 64x64
    image = tf.image.resize(image, (64, 64))
    return image, label

In [ ]:
train_ds_size = tf.data.experimental.cardinality(train_ds).numpy()
test_ds_size = tf.data.experimental.cardinality(test_ds).numpy()
validation_ds_size = tf.data.experimental.cardinality(validation_ds).numpy()
print("Train size:", train_ds_size)
print("Test size:", test_ds_size)
print("Validation size:", validation_ds_size)

In [ ]:
train_ds = (train_ds
            # pre-process the image
            .map(process_images)
            # shuffle the dataset
            .shuffle(buffer_size=train_ds_size)
            # batch dataset
            .batch(batch_size=32, drop_remainder=True))

# do the same for the test set and the validation set
test_ds = (test_ds
           .map(process_images)
           .shuffle(buffer_size=train_ds_size)
           .batch(batch_size=32, drop_remainder=True))

validation_ds = (validation_ds
                 .map(process_images)
                 .shuffle(buffer_size=train_ds_size)
                 .batch(batch_size=32, drop_remainder=True))

In [ ]:
# define the model
alex = keras.models.Sequential([
    keras.layers.Conv2D(filters=128, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(64,64,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3)),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(2048,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(2048,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10,activation='softmax')  
])

In [ ]:
alex.compile(loss='sparse_categorical_crossentropy', optimizer=tf.optimizers.SGD(learning_rate=0.001), metrics=['accuracy'])
alex.summary()

In [ ]:
alex.fit(train_ds,
         epochs=50,
         validation_data=validation_ds,
         validation_freq=1)

In [ ]:
plot_history(alex)

## Data Processing

In [ ]:
# encode the labels
y_train=np_utils.to_categorical(y_train)
y_validation=np_utils.to_categorical(y_validation)
y_test=np_utils.to_categorical(y_test)
print((x_train.shape,y_train.shape))
print((x_validation.shape,y_validation.shape))
print((x_test.shape,y_test.shape))

In [ ]:
# data augumentation
train_generator = ImageDataGenerator(rotation_range=2, 
                                    horizontal_flip=True,
                                    zoom_range=.1 )

val_generator = ImageDataGenerator(rotation_range=2, 
                                    horizontal_flip=True,
                                    zoom_range=.1)

test_generator = ImageDataGenerator(rotation_range=2, 
                                    horizontal_flip= True,
                                    zoom_range=.1)
train_generator.fit(x_train)
val_generator.fit(x_validation)
test_generator.fit(x_test)

## VGG-19

In [ ]:
# change the learning rate if the validation error does not reduce after a set number of epochs
lr_reducer = ReduceLROnPlateau(monitor='val_acc', # metric to be used
                       factor=.01, # reduce rate
                       patience=3, # number of epochs after which if there is no improvement in the val_acc, the learning rate is reduced
                       min_lr=1e-5) # the minimum learning rate 

In [ ]:
# import the pre-trained model
vgg = keras.applications.vgg19.VGG19(include_top=False,weights='imagenet',input_shape=(32,32,3),classes=y_train.shape[1])

In [ ]:
# finish building the model
vgg19 = keras.models.Sequential()
vgg19.add(vgg)
vgg19.add(keras.layers.Flatten())
vgg19.add(keras.layers.Dense(1024,activation=('relu'),input_dim=512))
vgg19.add(keras.layers.Dense(512,activation=('relu'))) 
vgg19.add(keras.layers.Dense(256,activation=('relu'))) 
vgg19.add(keras.layers.Dense(128,activation=('relu')))
vgg19.add(keras.layers.Dense(10,activation=('softmax'))) #This is the classification layer
vgg19.summary()

In [ ]:
# use momentum optimizer
sgd = tf.optimizers.SGD(learning_rate=.001,momentum=.9,nesterov=False)

In [ ]:
batch_size = 100
epochs = 50
vgg19.compile(optimizer=sgd,loss='categorical_crossentropy',metrics=['accuracy'])
vgg19.fit_generator(train_generator.flow(x_train,y_train,batch_size=batch_size),
                    epochs=epochs,
                    steps_per_epoch=x_train.shape[0]//batch_size,
                    validation_data=val_generator.flow(x_validation,y_validation,batch_size=batch_size),
                    validation_steps=250,
                    callbacks=[lr_reducer],verbose=1)

In [ ]:
f,ax=plt.subplots(2,1,figsize=(10,10)) 

# training loss and validation loss
ax[0].plot(vgg19.history.history['loss'],color='b',label='Training Loss')
ax[0].plot(vgg19.history.history['val_loss'],color='r',label='Validation Loss')

# training accuracy and validation accuracy
ax[1].plot(vgg19.history.history['accuracy'],color='b',label='Training  Accuracy')
ax[1].plot(vgg19.history.history['val_accuracy'],color='r',label='Validation Accuracy')

plt.legend()

## ResNet

### Define Residual Unit

In [ ]:
DefaultConv2D = partial(keras.layers.Conv2D, kernel_size=3, strides=1,
                        padding="SAME", use_bias=False)

class ResidualUnit(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layers = [
            DefaultConv2D(filters, strides=strides),
            keras.layers.BatchNormalization(),
            self.activation,
            DefaultConv2D(filters),
            keras.layers.BatchNormalization()]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                DefaultConv2D(filters, kernel_size=1, strides=strides),
                keras.layers.BatchNormalization()]

    def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z + skip_Z)

In [ ]:
# learning rate scheduler
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0,
                               patience=5, min_lr=0.5e-6)
# earlt stopping
early_stopper = EarlyStopping(min_delta=0.001, patience=10)

### ResNet-18

In [ ]:
# build ResNet-18
rn18 = keras.models.Sequential()
rn18.add(DefaultConv2D(64, kernel_size=7, strides=2,input_shape=(32,32,3)))
rn18.add(keras.layers.BatchNormalization())
rn18.add(keras.layers.Activation("relu"))
rn18.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding="SAME"))
prev_filters = 64
for filters in [64] * 2 + [128] * 2 + [256] * 2 + [512] * 2:
    strides = 1 if filters == prev_filters else 2
    rn18.add(ResidualUnit(filters, strides=strides))
    prev_filters = filters
rn18.add(keras.layers.GlobalAvgPool2D())
rn18.add(keras.layers.Flatten())
rn18.add(keras.layers.Dense(10, activation="softmax"))

In [ ]:
rn18.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# fit the model on the batches generated by the generator.
rn18.fit_generator(train_generator.flow(x_train, y_train, batch_size=32),
                   validation_data=val_generator.flow(x_validation,y_validation,batch_size=32),
                   steps_per_epoch = x_train.shape[0] // 32,
                   epochs=200,
                   callbacks=[lr_reducer, early_stopper])

### ResNet-34

In [ ]:
# build resnet-34
rn34 = keras.models.Sequential()
rn34.add(DefaultConv2D(64, kernel_size=7, strides=2,
                        input_shape=[224, 224, 3]))
rn34.add(keras.layers.BatchNormalization())
rn34.add(keras.layers.Activation("relu"))
rn34.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding="SAME"))
prev_filters = 64
for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
    strides = 1 if filters == prev_filters else 2
    rn34.add(ResidualUnit(filters, strides=strides))
    prev_filters = filters
rn34.add(keras.layers.GlobalAvgPool2D())
rn34.add(keras.layers.Flatten())
rn34.add(keras.layers.Dense(10, activation="softmax"))

In [ ]:
rn34.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
rn34.fit_generator(train_generator.flow(x_train, y_train, batch_size=32),
                   validation_data=val_generator.flow(x_validation,y_validation,batch_size=32),
                   steps_per_epoch = x_train.shape[0] // 32,
                   epochs=200,
                   callbacks=[lr_reducer, early_stopper])